In [12]:
import pandas as pd
import os

all_data = []

In [ ]:
# 1. กำหนดปีและชื่อไฟล์
# (สมมติว่าไฟล์ชื่อ survey_results_public_2020.csv ไล่ไปจนถึง 2025)

years = range(2020, 2026)

In [ ]:
# 2. สร้าง dict สำหรับแปลงชื่อคอลัมน์ปีใหม่ๆ (2021+) ให้กลับไปเป็นชื่อแบบปี 2020

column_mapping = {
    'LanguageHaveWorkedWith': 'LanguageWorkedWith',
    'LanguageWantToWorkWith': 'LanguageDesireNextYear',
    'DatabaseHaveWorkedWith': 'DatabaseWorkedWith',
    'DatabaseWantToWorkWith': 'DatabaseDesireNextYear',
    'PlatformHaveWorkedWith': 'PlatformWorkedWith',
    'PlatformWantToWorkWith': 'PlatformDesireNextYear',
    'WebframeHaveWorkedWith': 'WebframeWorkedWith',
    'WebframeWantToWorkWith': 'WebframeDesireNextYear',
    'MiscTechHaveWorkedWith': 'MiscTechWorkedWith',
    'MiscTechWantToWorkWith': 'MiscTechDesireNextYear'
}

In [ ]:
# 3. ระบุคอลัมน์ที่ต้องการ (Target Columns) เพื่อจะสร้างคอลัมน์นี้เพิ่มสำหรับ Dataframe อันใหม่

target_columns = [
    'year_of_survey',         
    'DevType',
    'LanguageWorkedWith', 'LanguageDesireNextYear',
    'WebframeWorkedWith', 'WebframeDesireNextYear',
    'DatabaseWorkedWith', 'DatabaseDesireNextYear',
    'MiscTechWorkedWith', 'MiscTechDesireNextYear',
    'PlatformWorkedWith', 'PlatformDesireNextYear'
]

In [14]:
# 4. Loop อ่านไฟล์ทีละปี

for year in years:
    file_path = f'survey_results_public_{year}.csv' # ตรวจสอบ path ไฟล์ให้ถูกต้อง
    
    if os.path.exists(file_path):
        print(f"Processing in {year}...")
        df = pd.read_csv(file_path, low_memory=False)
        
        # เปลี่ยนชื่อคอลัมน์ให้เป็นมาตรฐานเดียวกัน (Mapping)
        df.rename(columns=column_mapping, inplace=True)
        
        # เพิ่มคอลัมน์ระบุปีที่เก็บข้อมูลมา
        df['year_of_survey'] = year
        
        # เลือกเฉพาะคอลัมน์ที่มีอยู่จริง (ป้องกัน Error กรณีบางปีไม่มีบางคอลัมน์)
        available_cols = [col for col in target_columns if col in df.columns]
        
        # แจ้งเตือนถ้ามีคอลัมน์ไหนหายไปในปีก่อนๆ (นั่นคือ 2025 ที่เอาหัวข้อ 'MiscTechWorkedWith', 'MiscTechDesireNextYear' ออก โดยเราจะปล่อยให้เป็นค่า NaN)
        missing_cols = set(target_columns) - set(available_cols)
        if missing_cols:
            print(f"[WARNING]  year: {year} missing_cols: {missing_cols}")
            
        # ตัดข้อมูลมาเฉพาะที่ต้องการและเก็บเข้า List
        temp_df = df[available_cols].copy()
        all_data.append(temp_df)
    else:
        print(f"File not found. -> {file_path}")

Processing in 2020...
Processing in 2021...
Processing in 2022...
Processing in 2023...
Processing in 2024...
Processing in 2025...
[WARNING]  year: 2025 missing_cols: {'MiscTechWorkedWith', 'MiscTechDesireNextYear'}


In [18]:
# 5. รวมและส่งออก

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    
    print(f"\nData aggregation complete! \nTotal number of rows.: {len(final_df):,}")
    
    
    output_dir = '../../raw_data'           # ../.. หมายถึงถอยหลังออกมา 2 ชั้น (จาก Stack... -> src -> Root folder)
    os.makedirs(output_dir, exist_ok=True)  # สร้างโฟลเดอร์ raw_data เผื่อไว้ ถ้ายังไม่มี (กัน Error)
    
    # ตั้งชื่อไฟล์พร้อมระบุ Path
    output_filename = 'combined_SOsurvey_data.json'
    output_path = os.path.join(output_dir, output_filename)
    
    # ส่งออกเป็น JSON (orient='records' จะได้รูปแบบ Array of Objects)
    final_df.to_json(output_path, orient='records', indent=4)
    print("'combined_SOsurvey_data.json' file saved successfully.")
    
    # ดูตัวอย่างข้อมูล
    display(final_df.head())
else:
    print("[ERROR] No information was found. Please check the file name and file location.")


Data aggregation complete! 
Total number of rows.: 849,960
'combined_SOsurvey_data.json' file saved successfully.


,year_of_survey,DevType,LanguageWorkedWith,LanguageDesireNextYear,WebframeWorkedWith,WebframeDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear
0,2020,"Developer, desktop or enterprise applications;...",C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,ASP.NET;ASP.NET Core,ASP.NET Core,Elasticsearch;Microsoft SQL Server;Oracle,Microsoft SQL Server,.NET;.NET Core,.NET Core;Xamarin,Windows,Android;iOS;Kubernetes;Microsoft Azure;Windows
1,2020,"Developer, full-stack;Developer, mobile",JavaScript;Swift,Python;Swift,NaN,NaN,NaN,NaN,React Native,React Native;TensorFlow;Unity 3D,iOS,iOS;Kubernetes;Linux;MacOS
2,2020,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,NaN,HTML/CSS;Ruby;SQL,Java;Ruby;Scala,Ruby on Rails,Django;Ruby on Rails,MySQL;PostgreSQL;Redis;SQLite,MySQL;PostgreSQL,Ansible,Ansible;Chef,AWS;Docker;Linux;MacOS;Windows,Docker;Google Cloud Platform;Heroku;Linux;Windows
